In [5]:
import numpy as np
import matplotlib.pyplot as plt
import re
#from nltk.corpus import stopwords

In [6]:
%matplotlib inline

In [7]:
def read_data(file_name = './smsspamcollection/SMSSpamCollection.txt'):
    X = []
    y = []
    f = open(file_name)
    data = f.readlines()
    for line in data:
        line = line.strip().split('\t')
        y.append(line[0])
        X.append(line[1])
    return np.array(X),np.array(y)

def text_parser(X):
    res =[]
    for item in X:
        list0tokens = re.split(r' \W*',item)
        res.append(np.array([tok.lower() for tok in list0tokens if len(tok)>2]))
    return np.array(res)


def load_word_set():
    train_data= [
        ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], #[0,0,1,1,1......]
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
    ]
    train_y = [0, 1, 0, 1, 0, 1]
    return np.array(train_data),np.array(train_y)

def creat_vocab_list(train_X):
    vocab_list = set()
    for item in train_X:
        vocab_list = vocab_list.union(set(item))
    return list(vocab_list)

def _words2vec(vocab_list,x):
    input_set = set(x)
    return_vec = np.zeros(len(vocab_list))
    for i in range(len(vocab_list)):
        if vocab_list[i] in input_set:
            return_vec[i] = 1    
    return return_vec

def words2vec(vocab_list,X):
    return_mat = []
    for item in X:
        return_mat.append(_words2vec(vocab_list,item))
    return np.array(return_mat)


def fit(vocab_list,X,y):
    target_list = np.unique(y)
    p_word_mat = []
    p_c = []
    for item in target_list:
        inx = y==item
        target_X = X[inx]
        target_y = y[inx]
        p_c.append(len(target_y)/len(y))
        p_word_mat.append((np.sum(target_X,axis=0)+1)/(len(target_X)+1)) #避免分子为零
    return np.array(p_c),np.array(p_word_mat)

def _predict(vocab_list,p_c,p_word_mat,x):
    x_vec = _words2vec(vocab_list,x)
    res = np.log(p_c)+np.sum(np.log(p_word_mat)*x_vec,axis=1)
    return np.argmax(res)

In [162]:
word_X, word_y = load_word_set()

vocab_list = creat_vocab_list(X)

train_X = words2vec(vocab_list,X)

train_y = y

In [177]:
p_c,p_word_mat=fit(vocab_list,train_X,train_y)

In [178]:
_predict(vocab_list,p_c,p_word_mat,word_X[0])

1

In [147]:
res

array([], shape=(5574, 0), dtype=float64)

In [159]:
np.array([re.split(r' \W*',X[40])]).shape

(1, 17)

## NaiveBayes

In [8]:
from naive_bayes import NaiveBayesClassifier
from model_selection import train_test_split

nbc = NaiveBayesClassifier()

In [9]:
X,y  = read_data()

X = text_parser(X)
y[y =='spam']=1
y[y=='ham']=0
y = np.array(y,dtype=np.int)

In [10]:
y.sum()/len(y)

0.1340150699677072

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_ratio=0.3,seed=None)

In [24]:
nbc.fit(X_train,y_train)

In [25]:
predictions = nbc.predict(X_test)

In [14]:
predictions

array([0, 0, 1, ..., 1, 0, 1])

In [27]:
from metric import confusion_matrix,recall_score,precision_score,accuracy_score

In [26]:
precision_score(y_test,predictions)

0.4094076655052265

In [28]:
accuracy_score(y_test,predictions)

0.7954545454545454

In [29]:
from collections import Counter

In [34]:
y_rand = np.random.randint(0,2,len(y_test))

0.12835472578763127